In [1]:
!pip install -U -q PyDrive

    100% |████████████████████████████████| 993kB 15.3MB/s 


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
#downloaded = drive.CreateFile({'id':'1kO1y5CRbabnobaZ4RrOBRy7vLmBq4tAn'}) 
#downloaded.GetContentFile('train.csv')

### Using Glove Embedding to reduce the number of features for  predictions. 

Reference link: https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

About Different Embeddings: https://www.kaggle.com/sbongo/do-pretrained-embeddings-give-you-the-extra-edge

In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [0]:
all_df = pd.read_csv("/content/drive/My Drive/Quora_Spam_Classification/train.csv")

In [8]:
all_df.shape

(1306122, 3)

In [0]:
train_df, test_df= train_test_split(all_df, test_size=0.2)

In [0]:
train_df, val_df= train_test_split(train_df, test_size=0.1)

In [11]:
train_df.head()

,qid,question_text,target
1106089,d8c1930a77aee5002e1f,How do you deal with yourself and other people...,0
829383,a28c31cce0377727bf7d,Which books should I study for SSC CGL?,0
1232401,f1813b68761c10a79176,How can I move pics from thumbnail folder to D...,0
340374,42ad816b68d2ad08284e,What happened to John Hart after he signed the...,0
117848,17115e6b8b3564eff18b,How many continental plates are there?,0


In [12]:
train_df.shape

(940407, 3)

In [13]:
train_df[train_df['target']==1].head()

,qid,question_text,target
477796,5d900b54734f0dfd2f78,How do fundamentalists rationalize murdering d...,1
970393,be20bbd330b79165465c,"As an apatheist, why is it that I find the mor...",1
60585,0be5b3acc9c1094fbd7b,Why is Europe so ugly? Why do Europeans feel t...,1
20933,041ab2d323d554a8bbf9,What do I do to be more confident about showin...,1
900825,b08037e705d998f56be6,"Why doesn’t the US invade North Korea, steal t...",1


In [0]:
#Tokenization
from keras.preprocessing.text import  Tokenizer

## some config values 
embed_size = 300 # how big is each word vector
max_features = 50000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 70 # max number of words in a question to use

##Filling up missing values
train_X = train_df["question_text"].fillna("_na_").values
val_X = val_df["question_text"].fillna("_na_").values
test_X = test_df["question_text"].fillna("_na_").values

## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)
test_X = tokenizer.texts_to_sequences(test_X)

In [0]:
## Padding sentences with max length
from keras.preprocessing.sequence import pad_sequences
train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)

In [0]:
## transforming target into array
train_y = train_df['target'].values
val_y = val_df['target'].values
test_y = test_df['target'].values

### From viewing the data, spam questions are filtered with abusive words, and politics etc.

In [19]:
#downloaded = drive.CreateFile({'id':'1fVwAHGvIslyZwKNsvg0ULHTp5R6iTZQD'}) 
#downloaded.GetContentFile('glove.840B.300d.txt')
embeddings_index = {}
f = open('/content/drive/My Drive/Quora_Spam_Classification/glove.840B.300d.txt')
for line in tqdm(f):
    values = line.split(" ")
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

2196017it [04:13, 8658.33it/s] 

Found 2196016 word vectors.


In [0]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.optimizers import Adam
from keras.layers import Embedding

In [0]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [32]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Conv1D(64, kernel_size=4,activation='relu',padding='same')(x)
x = MaxPooling1D()(x)
x = Conv1D(64, activation='relu',kernel_size=4,padding='same')(x)
x = Dense(24, activation='relu')(x)
x = Dropout(0.1)(x)
x = GlobalAveragePooling1D()(x)
x = Dense(16, activation='relu')(x)
x = Dropout(0.1)(x)
x = Dense(1, activation='softmax')(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 70)                0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 70, 300)           15000000  
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 70, 64)            76864     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 35, 64)            0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 35, 64)            16448     
_________________________________________________________________
dense_4 (Dense)              (None, 35, 24)            1560      
_________________________________________________________________
dropout_3 (Dropout)          (None, 35, 24)            0         
__________

In [34]:
model.fit(train_X, train_y, batch_size=216, epochs=5, validation_data=(val_X,val_y))

Instructions for updating:
Use tf.cast instead.
Train on 940407 samples, validate on 104490 samples
Epoch 1/5
940407/940407 [==============================] - 156s 166us/step - loss: 14.9550 - acc: 0.0619 - val_loss: 14.9580 - val_acc: 0.0617
Epoch 2/5
940407/940407 [==============================] - 151s 161us/step - loss: 14.9550 - acc: 0.0619 - val_loss: 14.9580 - val_acc: 0.0617
Epoch 3/5
940407/940407 [==============================] - 151s 160us/step - loss: 14.9550 - acc: 0.0619 - val_loss: 14.9580 - val_acc: 0.0617
Epoch 4/5
940407/940407 [==============================] - 151s 160us/step - loss: 14.9550 - acc: 0.0619 - val_loss: 14.9580 - val_acc: 0.0617
Epoch 5/5
940407/940407 [==============================] - 152s 162us/step - loss: 14.9550 - acc: 0.0619 - val_loss: 14.9580 - val_acc: 0.0617


In [41]:
from sklearn.metrics import f1_score
pred_glove_val_y = model.predict([val_X], batch_size=1024, verbose=1)
f1_score(pred_glove_val_y,val_y)

104490/104490 [==============================] - 2s 20us/step


0.11631302842926934

In [39]:
pred_glove_test_y = model.predict([test_X], batch_size=1024, verbose=1)

261225/261225 [==============================] - 5s 19us/step


In [42]:
 f1_score(pred_glove_test_y,test_y)

0.11619077151624198

In [43]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Conv1D(64, kernel_size=4,activation='relu',padding='same')(x)
x = MaxPooling1D()(x)
x = Conv1D(64, activation='relu',kernel_size=4,padding='same')(x)
x = Dense(24, activation='relu')(x)
x = Dropout(0.1)(x)
x = GlobalAveragePooling1D()(x)
x = Dense(16, activation='relu')(x)
x = Dropout(0.1)(x)
x = Dense(1, activation='sigmoid')(x)
model1 = Model(inputs=inp, outputs=x)
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 70)                0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 70, 300)           15000000  
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 70, 64)            76864     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 35, 64)            0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 35, 64)            16448     
_________________________________________________________________
dense_7 (Dense)              (None, 35, 24)            1560      
_________________________________________________________________
dropout_5 (Dropout)          (None, 35, 24)            0         
__________

In [44]:
model.fit(train_X, train_y, batch_size=512, epochs=3, validation_data=(val_X,val_y))

Train on 940407 samples, validate on 104490 samples
Epoch 1/3
940407/940407 [==============================] - 93s 99us/step - loss: 14.9550 - acc: 0.0619 - val_loss: 14.9580 - val_acc: 0.0617
Epoch 2/3
940407/940407 [==============================] - 92s 97us/step - loss: 14.9550 - acc: 0.0619 - val_loss: 14.9580 - val_acc: 0.0617
Epoch 3/3
940407/940407 [==============================] - 92s 98us/step - loss: 14.9550 - acc: 0.0619 - val_loss: 14.9580 - val_acc: 0.0617


In [55]:
pred_glove_val_y = model1.predict([val_X], batch_size=1024, verbose=1)
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, f1_score(val_y, (pred_glove_val_y>thresh).astype(int))))

104490/104490 [==============================] - 2s 20us/step
F1 score at threshold 0.1 is 0.11631302842926934
F1 score at threshold 0.11 is 0.11631302842926934
F1 score at threshold 0.12 is 0.11631302842926934
F1 score at threshold 0.13 is 0.11631302842926934
F1 score at threshold 0.14 is 0.11631302842926934
F1 score at threshold 0.15 is 0.11631302842926934
F1 score at threshold 0.16 is 0.11631302842926934
F1 score at threshold 0.17 is 0.11631302842926934
F1 score at threshold 0.18 is 0.11631302842926934
F1 score at threshold 0.19 is 0.11631302842926934
F1 score at threshold 0.2 is 0.11631302842926934
F1 score at threshold 0.21 is 0.11631302842926934
F1 score at threshold 0.22 is 0.11631302842926934
F1 score at threshold 0.23 is 0.11631302842926934
F1 score at threshold 0.24 is 0.11631302842926934
F1 score at threshold 0.25 is 0.11631302842926934
F1 score at threshold 0.26 is 0.11631302842926934
F1 score at threshold 0.27 is 0.11631302842926934
F1 score at threshold 0.28 is 0.11631302

In [56]:
pred_glove_test_y = model1.predict([test_X], batch_size=1024, verbose=1) 
pred_glove_test_y

261225/261225 [==============================] - 5s 19us/step


array([[0.5560361 ],
       [0.5499636 ],
       [0.57250476],
       ...,
       [0.56616586],
       [0.55861807],
       [0.56827074]], dtype=float32)

In [0]:
from keras.layers import LSTM

In [60]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = LSTM(200, dropout_U = 0.2, dropout_W = 0.2)(x)
x = Dense(1, activation='sigmoid')(x)
model2 = Model(inputs=inp, outputs=x)
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model2.summary())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(200, dropout=0.2, recurrent_dropout=0.2)`
  This is separate from the ipykernel package so we can avoid doing imports until


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 70)                0         
_________________________________________________________________
embedding_7 (Embedding)      (None, 70, 300)           15000000  
_________________________________________________________________
lstm_3 (LSTM)                (None, 200)               400800    
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 201       
Total params: 15,401,001
Trainable params: 15,401,001
Non-trainable params: 0
_________________________________________________________________
None


In [62]:
model2.fit(train_X, train_y, validation_data=(val_X, val_y),
          epochs=3, batch_size=512, verbose=1)

Train on 940407 samples, validate on 104490 samples
Epoch 1/3
940407/940407 [==============================] - 474s 504us/step - loss: 0.1159 - acc: 0.9544 - val_loss: 0.1058 - val_acc: 0.9578
Epoch 2/3
940407/940407 [==============================] - 473s 503us/step - loss: 0.0956 - acc: 0.9615 - val_loss: 0.1025 - val_acc: 0.9591
Epoch 3/3
940407/940407 [==============================] - 473s 503us/step - loss: 0.0834 - acc: 0.9664 - val_loss: 0.1062 - val_acc: 0.9585


In [63]:
pred_glove_val_y = model2.predict([val_X], batch_size=1024, verbose=1)
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, f1_score(val_y, (pred_glove_val_y>thresh).astype(int))))

104490/104490 [==============================] - 13s 126us/step
F1 score at threshold 0.1 is 0.5916553382829657
F1 score at threshold 0.11 is 0.5995408680796541
F1 score at threshold 0.12 is 0.60686503835065
F1 score at threshold 0.13 is 0.613153232949513
F1 score at threshold 0.14 is 0.6192676056338029
F1 score at threshold 0.15 is 0.6252576139226014
F1 score at threshold 0.16 is 0.6296554124608422
F1 score at threshold 0.17 is 0.6344161958568738
F1 score at threshold 0.18 is 0.6385010144408642
F1 score at threshold 0.19 is 0.6429348483017043
F1 score at threshold 0.2 is 0.6475910980679873
F1 score at threshold 0.21 is 0.6507916872835231
F1 score at threshold 0.22 is 0.6527118220444889
F1 score at threshold 0.23 is 0.6551811182632841
F1 score at threshold 0.24 is 0.6572704081632654
F1 score at threshold 0.25 is 0.6599652039435531
F1 score at threshold 0.26 is 0.6606700560552731
F1 score at threshold 0.27 is 0.6627172195892574
F1 score at threshold 0.28 is 0.6624161519559009
F1 score a

In [67]:
pred_glove_test_y = model2.predict([test_X], batch_size=1024, verbose=1) 
pred_glove_test_y

261225/261225 [==============================] - 32s 123us/step


array([[0.04962832],
       [0.00091454],
       [0.00386915],
       ...,
       [0.01019022],
       [0.18808252],
       [0.00203487]], dtype=float32)

In [68]:
f1_score(test_y, (pred_glove_test_y>0.37).astype(int))

0.6698954004871759